In [ ]:
!pip install geopandas --user -q

In [1]:
import os
import geopyspark as gps

from pyspark import SparkContext

from overlay import overlay, layers, boundaries, constants, health_resources

In [2]:
conf = gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '12G')
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

*Houston*

In [ ]:
b = os.path.join(constants.BOUNDARY_OUTPUT_DIR, 'houston.shp')
u = os.path.join(constants.NOAA_URL_BASE, 'TX/TX_HGX_slr_data_dist.zip')
cm = gps.ColorMap.build(breaks=constants.VALUES, colors = constants.COLORS)
hou = overlay.overlay_analysis(M, 'hou', b, u, constants.NATL_STORM_SURGE_5, color_map=cm)

*New York*

In [ ]:
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/NY/NY_Metro_slr_data_dist.zip'
b = '/home/hadoop/notebooks/data/write/boundaries/newyork.shp'
ny = overlay.overlay_analysis(M, 'ny', b, u)

*Suffolk County (Long Island)*

In [ ]:
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/NY/NY_Suffolk_slr_data_dist.zip'
b = boundaries.get_county_study_area('suffolk', 36, 'suffolk.shp')
suffolk = overlay.overlay_analysis(M, 'suffolk', b, u)

*Tampa*

In [ ]:
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/FL/FL_TBW_slr_data_dist.zip'
b = boundaries.get_msa_study_area('tampa','tampa_msa.shp')
tampa = overlay.overlay_analysis(M, 'tampa', b, u)

In [8]:
import geopandas as gpd
f = gpd.read_file('/home/hadoop/notebooks/data/read/overlay-layers/flood/Fld_Haz_ar.shp')

In [9]:
import pandas as pd
ff = pd.DataFrame(f)

In [10]:
ff.head()

,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,...,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,GFID,SHAPE_Leng,SHAPE_Area,geometry
0,48057C,1.1.1.0,48057C_101,NP,VE,None,T,13.0,NAVD88,-9999.0,...,None,None,-9999.0,-9999.0,None,48057C_STUDY1,20140930,0.037721,9.591310e-06,POLYGON ((-96.41820397784505 28.68983344243782...
1,48057C,1.1.1.0,48057C_102,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,48057C_STUDY1,20140930,0.013807,8.697384e-06,POLYGON ((-96.48332642396065 28.70513590536899...
2,48057C,1.1.1.0,48057C_103,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,48057C_STUDY1,20140930,0.276885,1.436534e-03,POLYGON ((-96.53228897732873 28.68057395479269...
3,01063C,1.1.1.0,01063C_1026,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,01063C_STUDY2,None,0.002934,5.641283e-07,POLYGON ((-87.83938510452873 32.60835435512706...
4,01063C,1.1.1.0,01063C_1181,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,01063C_STUDY2,None,0.001871,2.098302e-07,POLYGON ((-88.14937782522617 32.95524740858554...


### step by step

In [ ]:
hou = overlay.Overlay('hou','/home/hadoop/notebooks/data/read/boundaries/houston.shp')

In [ ]:
M.set_center(hou.study_area['centroid_wgs']['x'], hou.study_area['centroid_wgs']['y'], 9)

*Prep Input Layer Datasets*

In [ ]:
flood = '/home/hadoop/notebooks/data/read/overlay-layers/flood/Fld_Haz_ar.shp'
sea = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/TX/TX_HGX_slr_data_dist.zip'
storm = '/home/hadoop/notebooks/data/read/overlay-layers/storm-surge/US_Category5_MOM_Inundation_HighTide.tif'

input_data = hou.prep_data(flood, sea, storm)

*Storm Surge*

In [ ]:
hou.storm_surge = layers.storm_surge(hou, input_data['storm_surge'])

In [ ]:
overlay.map_layer(M, hou.storm_surge)

In [ ]:
hou.storm_surge.get_histogram().bin_counts()

*Sea Level Rise*

In [ ]:
hou.sea_level_rise = layers.sea_level_rise(hou, input_data['sea_level'])

In [ ]:
hou.sea_level_rise.get_histogram().bin_counts()

In [ ]:
cm = gps.ColorMap.build(breaks=[1, 5], colors=[0xD28170FF, 0x526095FF])

In [ ]:
overlay.map_layer(M, hou.sea_level_rise, cm)

*Flood Hazard*

In [ ]:
hou.flood = layers.flood_hazard(hou, input_data['flood'])

In [ ]:
hou.flood.get_histogram().bin_counts()

In [ ]:
overlay.map_layer(M, hou.flood, cm)

In [ ]:
ov = hou.flood + hou.sea_level_rise + hou.storm_surge - 3

In [ ]:
ov.get_histogram().bin_counts()

In [ ]:
overlay.map_layer(M, ov)

# Study Area

In [ ]:
base_gpd = gpd.read_file("data/read/boundaries/houston.shp")
base_poly = [base_gpd.geometry[0]]

In [ ]:
base_raster = gps.rasterize(base_poly, 4326, 12, 1, gps.CellType.INT8)

In [ ]:
base_tiled = base_raster.tile_to_layout(gps.GlobalLayout(), 3857)

#### Visualize Study Area

In [ ]:
mapp(base_tiled)

# Storm Surge

In [ ]:
storm_raster = gps.geotiff.get(gps.LayerType.SPATIAL,
                    '/home/hadoop/notebooks/data/write/overlay-layers/storm-surge/houston_flood_category5.tif',
                     max_tile_size=128,
                     num_partitions=32)

In [ ]:
rmap = {}
for x in range(2, 24, 2):
    rmap[x + 1] = int(x / 2)

storm_reclass = surge_tiled.reclassify(value_map=rmap,
                                    data_type=int,
                                    replace_nodata_with=1,
                                    classification_strategy=gps.ClassificationStrategy.LESS_THAN_OR_EQUAL_TO)

In [ ]:
storm_tiled = storm_reclass.tile_to_layout(base_tiled.layer_metadata)

#### Visualize Storm Surge

In [ ]:
mapp(storm_tiled)

#### Add Storm Surge and Houston Area

In [ ]:
storm_overlay = storm_tiled * base_tiled 

In [ ]:
mapp(storm_overlay)

# Sea Level Rise

In [ ]:
gdb = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb"

In [ ]:
sea_poly = []
with fiona.open(gdb, layer='TX_HGX_slr_5ft') as source:
    slr_crs = source.crs['init']
    for p in source:
        sea_poly += MultiPolygon(shape(p['geometry']))

In [ ]:
sea_raster = gps.rasterize(slr,
                           4326,
                           12,
                           5,
                           gps.CellType.INT8)

In [ ]:
sea_reclass = sea_raster.reclassify(value_map={5: 5},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

In [ ]:
sea_tiled = sea_reclass.tile_to_layout(gps.GlobalLayout(), 3857)

#### Union And Aggregate Sea Level and Houston

In [ ]:
sea_unioned_base = gps.geotrellis.union(layers=[base_tiled, sea_tiled])

In [ ]:
sea_aggregated_base = sea_unioned_base.aggregate_by_cell(operation=gps.geotrellis.constants.Operation.MAX)

In [ ]:
sea_overlay = sea_aggregated_base.reclassify(value_map={5: 5, 1:1},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

#### Visualizing Sea Level Rise

In [ ]:
mapp(sea_overlay)

# Flood Hazard

In [ ]:
flood_poly = data.read_from_shp('/home/hadoop/notebooks/data/write/overlay-layers/flood/Fld_Haz_ar_wm_houston.shp')

In [ ]:
flood_raster = gps.rasterize(geoms=flood_poly,
                                 crs = 3857,
                                 zoom=12,
                                 fill_value = 5,
                                 options = gps.RasterizerOptions(
                                     includePartial=True, 
                                     sampleType='PixelIsArea'),
                                 cell_type = gps.CellType.INT8,
                                 num_partitions = 50)

In [ ]:
flood_reclass = flood_raster.reclassify(value_map={5: 5},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

In [ ]:
flood_tiled = flood_reclass.tile_to_layout(gps.GlobalLayout(), 3857)

#### Union And Aggregate Flood Hazard and Houston

In [ ]:
flood_unioned_base = gps.geotrellis.union(layers=[flood_tiled, base_tiled])

In [ ]:
flood_aggregated_base = flood_unioned_base.aggregate_by_cell(operation=gps.geotrellis.constants.Operation.MAX)

In [ ]:
flood_overlay = flood_aggregated_base.reclassify(value_map={5: 5, 1:1},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

#### Visualizing Sea Level Rise

In [ ]:
mapp(flood_overlay)

# Overlay

In [ ]:
overlay = flood_overlay + sea_overlay + storm_overlay - 3

In [ ]:
mapp(overlay)